In [1]:
import os
import sys
import numpy as np
import h5py
root = os.path.dirname(os.path.abspath(os.curdir))
sys.path.append(root)

import torch
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import DataLoader
from torch.utils.data import Subset

from predify.utils.training import train_pcoders, eval_pcoders

from networks_2022 import BranchedNetwork
from data.ReconstructionTrainingDataset import CleanSoundsDataset
from data.ReconstructionTrainingDataset import NoisySoundsDataset

# Specify Network to train
TODO: This should be converted to a script that accepts arguments for which network to train

In [2]:
snrs = ['neg6', 'neg3', '0', '3']
pnet_names = ['pnet_snr-6', 'pnet_snr-3', 'pnet_snr0', 'pnet_snr3']

# Parameters

In [ ]:
for snr, pnet_name in zip(snrs, pnet_names):
    _train_datafile = 'hyperparameter_pooled_training_dataset_random_order_noNulls'
    SoundsDataset = NoisySoundsDataset
    dset_kwargs = {'snr': snr}
    from pbranchednetwork_all import PBranchedNetwork_AllSeparateHP
    PNetClass = PBranchedNetwork_AllSeparateHP

    DEVICE = 'cuda:0' if torch.cuda.is_available() else 'cpu'
    print(f'Device: {DEVICE}')
    BATCH_SIZE = 50
    NUM_WORKERS = 2
    PIN_MEMORY = True
    NUM_EPOCHS = 70

    lr = 1E-5
    engram_dir = '/mnt/smb/locker/abbott-locker/hcnn/'
    checkpoints_dir = f'{engram_dir}checkpoints/'
    tensorboard_dir = f'{engram_dir}tensorboard/'
    train_datafile = f'{engram_dir}{_train_datafile}.hdf5'

    net = BranchedNetwork()
    net.load_state_dict(torch.load(f'{engram_dir}networks_2022_weights.pt'))

    pnet = PNetClass(net, build_graph=True)
    pnet.eval()
    pnet.to(DEVICE)
    optimizer = torch.optim.Adam(
        [{'params':getattr(pnet,f"pcoder{x+1}").pmodule.parameters(), 'lr':lr} for x in range(pnet.number_of_pcoders)],
        weight_decay=5e-4)

    train_dataset = SoundsDataset(
        train_datafile, subset=.9, **dset_kwargs)
    test_dataset = SoundsDataset(
        train_datafile, subset=.9,
        train = False, **dset_kwargs
        )

    train_loader = DataLoader(
        train_dataset, batch_size=BATCH_SIZE, shuffle=False,
        num_workers=NUM_WORKERS, pin_memory=PIN_MEMORY
        )
    eval_loader = DataLoader(
        test_dataset, batch_size=BATCH_SIZE, shuffle=False,
        num_workers=NUM_WORKERS, pin_memory=PIN_MEMORY
        )

    checkpoint_path = os.path.join(checkpoints_dir, f"{pnet_name}")
    if not os.path.exists(checkpoint_path):
        os.makedirs(checkpoint_path)
    checkpoint_path = os.path.join(checkpoint_path, pnet_name + '-{epoch}-{type}.pth')

    # summarywriter
    from torch.utils.tensorboard import SummaryWriter
    tensorboard_path = os.path.join(tensorboard_dir, f"{pnet_name}")
    if not os.path.exists(tensorboard_path):
        os.makedirs(tensorboard_path)
    sumwriter = SummaryWriter(tensorboard_path, filename_suffix=f'')

    loss_function = torch.nn.MSELoss()
    for epoch in range(1, NUM_EPOCHS+1):
        train_pcoders(pnet, optimizer, loss_function, epoch, train_loader, DEVICE, sumwriter)
        eval_pcoders(pnet, loss_function, epoch, eval_loader, DEVICE, sumwriter)

        # save checkpoints every 5 epochs
        if epoch % 5 == 0:
            torch.save(pnet.state_dict(), checkpoint_path.format(epoch=epoch, type='regular'))

Device: cuda:0


/share/issa/users/es3773/hallucnn/src/models/layers.py:78: UserWarning: Inconsistent tf pad calculation in ConvLayer.
  warnings.warn('Inconsistent tf pad calculation in ConvLayer.')
/share/issa/users/es3773/hallucnn/src/models/layers.py:173: UserWarning: Inconsistent tf pad calculation: 0, 1
  warnings.warn(f'Inconsistent tf pad calculation: {pad_left}, {pad_right}')
/home/es3773/.conda/envs/hcnn/lib/python3.6/site-packages/torch/nn/functional.py:3680: UserWarning: The default behavior for interpolate/upsample with float scale_factor changed in 1.6.0 to align with other frameworks/libraries, and now uses scale_factor directly, instead of relying on the computed output size. If you wish to restore the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  "The default behavior for interpolate/upsample with float scale_factor changed "


Training Epoch: 1 [50/13500]	Loss: 95908.8750
Training Epoch: 1 [100/13500]	Loss: 92091.5469
Training Epoch: 1 [150/13500]	Loss: 88681.8516
Training Epoch: 1 [200/13500]	Loss: 84799.6641
Training Epoch: 1 [250/13500]	Loss: 80978.5391
Training Epoch: 1 [300/13500]	Loss: 78071.8359
Training Epoch: 1 [350/13500]	Loss: 75638.1875
Training Epoch: 1 [400/13500]	Loss: 71907.4062
Training Epoch: 1 [450/13500]	Loss: 68443.6328
Training Epoch: 1 [500/13500]	Loss: 66457.0625
Training Epoch: 1 [550/13500]	Loss: 63745.8086
Training Epoch: 1 [600/13500]	Loss: 60915.1719
Training Epoch: 1 [650/13500]	Loss: 58007.0391
Training Epoch: 1 [700/13500]	Loss: 57634.0039
Training Epoch: 1 [750/13500]	Loss: 52949.9336
Training Epoch: 1 [800/13500]	Loss: 51815.1133
Training Epoch: 1 [850/13500]	Loss: 49306.4375
Training Epoch: 1 [900/13500]	Loss: 45973.4844
Training Epoch: 1 [950/13500]	Loss: 44763.1875
Training Epoch: 1 [1000/13500]	Loss: 43941.3516
Training Epoch: 1 [1050/13500]	Loss: 40997.6445
Training Epo

Training Epoch: 1 [8650/13500]	Loss: 13205.1875
Training Epoch: 1 [8700/13500]	Loss: 13262.8867
Training Epoch: 1 [8750/13500]	Loss: 12931.3613
Training Epoch: 1 [8800/13500]	Loss: 12956.1025
Training Epoch: 1 [8850/13500]	Loss: 12558.4326
Training Epoch: 1 [8900/13500]	Loss: 13189.6826
Training Epoch: 1 [8950/13500]	Loss: 13303.2412
Training Epoch: 1 [9000/13500]	Loss: 12785.3740
Training Epoch: 1 [9050/13500]	Loss: 12888.8555
Training Epoch: 1 [9100/13500]	Loss: 13224.3936
Training Epoch: 1 [9150/13500]	Loss: 13017.5430
Training Epoch: 1 [9200/13500]	Loss: 12529.8350
Training Epoch: 1 [9250/13500]	Loss: 12816.5205
Training Epoch: 1 [9300/13500]	Loss: 13279.3721
Training Epoch: 1 [9350/13500]	Loss: 13538.3291
Training Epoch: 1 [9400/13500]	Loss: 12641.2324
Training Epoch: 1 [9450/13500]	Loss: 12332.1191
Training Epoch: 1 [9500/13500]	Loss: 12523.5596
Training Epoch: 1 [9550/13500]	Loss: 12805.7041
Training Epoch: 1 [9600/13500]	Loss: 13002.4180
Training Epoch: 1 [9650/13500]	Loss: 124

Training Epoch: 2 [3650/13500]	Loss: 9773.3516
Training Epoch: 2 [3700/13500]	Loss: 9583.1143
Training Epoch: 2 [3750/13500]	Loss: 9739.9102
Training Epoch: 2 [3800/13500]	Loss: 9980.4629
Training Epoch: 2 [3850/13500]	Loss: 9890.6826
Training Epoch: 2 [3900/13500]	Loss: 9416.0752
Training Epoch: 2 [3950/13500]	Loss: 9743.9053
Training Epoch: 2 [4000/13500]	Loss: 9402.0596
Training Epoch: 2 [4050/13500]	Loss: 9651.7002
Training Epoch: 2 [4100/13500]	Loss: 9632.4121
Training Epoch: 2 [4150/13500]	Loss: 9938.3184
Training Epoch: 2 [4200/13500]	Loss: 9633.9258
Training Epoch: 2 [4250/13500]	Loss: 9528.5381
Training Epoch: 2 [4300/13500]	Loss: 9419.1104
Training Epoch: 2 [4350/13500]	Loss: 9524.7285
Training Epoch: 2 [4400/13500]	Loss: 9404.9414
Training Epoch: 2 [4450/13500]	Loss: 9581.7822
Training Epoch: 2 [4500/13500]	Loss: 9220.6357
Training Epoch: 2 [4550/13500]	Loss: 9093.5176
Training Epoch: 2 [4600/13500]	Loss: 9221.5020
Training Epoch: 2 [4650/13500]	Loss: 9678.8555
Training Epoc

...

Training Epoch: 8 [9500/13500]	Loss: 2732.5061
Training Epoch: 8 [9550/13500]	Loss: 2756.7817
Training Epoch: 8 [9600/13500]	Loss: 2728.4631
Training Epoch: 8 [9650/13500]	Loss: 2691.5305
Training Epoch: 8 [9700/13500]	Loss: 2833.3174
Training Epoch: 8 [9750/13500]	Loss: 2749.9414
Training Epoch: 8 [9800/13500]	Loss: 2674.2620
Training Epoch: 8 [9850/13500]	Loss: 2727.5176
Training Epoch: 8 [9900/13500]	Loss: 2636.2253
Training Epoch: 8 [9950/13500]	Loss: 2796.4951
Training Epoch: 8 [10000/13500]	Loss: 2741.1013
Training Epoch: 8 [10050/13500]	Loss: 2740.6272
Training Epoch: 8 [10100/13500]	Loss: 2724.1431
Training Epoch: 8 [10150/13500]	Loss: 2800.3333
Training Epoch: 8 [10200/13500]	Loss: 2698.8389
Training Epoch: 8 [10250/13500]	Loss: 2705.9980
Training Epoch: 8 [10300/13500]	Loss: 2684.8994
Training Epoch: 8 [10350/13500]	Loss: 2734.2063
Training Epoch: 8 [10400/13500]	Loss: 2804.5647
Training Epoch: 8 [10450/13500]	Loss: 2789.0359
Training Epoch: 8 [10500/13500]	Loss: 2765.9126
Tr

Training Epoch: 9 [4650/13500]	Loss: 2680.6389
Training Epoch: 9 [4700/13500]	Loss: 2598.5574
Training Epoch: 9 [4750/13500]	Loss: 2626.4741
Training Epoch: 9 [4800/13500]	Loss: 2575.9080
Training Epoch: 9 [4850/13500]	Loss: 2568.3779
Training Epoch: 9 [4900/13500]	Loss: 2611.6873
Training Epoch: 9 [4950/13500]	Loss: 2524.0774
Training Epoch: 9 [5000/13500]	Loss: 2511.9617
Training Epoch: 9 [5050/13500]	Loss: 2543.3062
Training Epoch: 9 [5100/13500]	Loss: 2623.7937
Training Epoch: 9 [5150/13500]	Loss: 2532.1721
Training Epoch: 9 [5200/13500]	Loss: 2511.4619
Training Epoch: 9 [5250/13500]	Loss: 2559.8687
Training Epoch: 9 [5300/13500]	Loss: 2539.1655
Training Epoch: 9 [5350/13500]	Loss: 2530.5051
Training Epoch: 9 [5400/13500]	Loss: 2608.9524
Training Epoch: 9 [5450/13500]	Loss: 2587.4231
Training Epoch: 9 [5500/13500]	Loss: 2602.7664
Training Epoch: 9 [5550/13500]	Loss: 2510.4644
Training Epoch: 9 [5600/13500]	Loss: 2536.1560
Training Epoch: 9 [5650/13500]	Loss: 2619.7312
Training Epoc

Training Epoch: 9 [13300/13500]	Loss: 2395.9692
Training Epoch: 9 [13350/13500]	Loss: 2457.1318
Training Epoch: 9 [13400/13500]	Loss: 2370.2734
Training Epoch: 9 [13450/13500]	Loss: 2438.8840
Training Epoch: 9 [13500/13500]	Loss: 2484.5833
Training Epoch: 9 [1499/1499]	Loss: 2424.9558
Training Epoch: 10 [50/13500]	Loss: 2447.2761
Training Epoch: 10 [100/13500]	Loss: 2409.1567
Training Epoch: 10 [150/13500]	Loss: 2377.2871
Training Epoch: 10 [200/13500]	Loss: 2390.9792
Training Epoch: 10 [250/13500]	Loss: 2460.6719
Training Epoch: 10 [300/13500]	Loss: 2430.4329
Training Epoch: 10 [350/13500]	Loss: 2457.6050
Training Epoch: 10 [400/13500]	Loss: 2449.8616
Training Epoch: 10 [450/13500]	Loss: 2395.2173
Training Epoch: 10 [500/13500]	Loss: 2379.1592
Training Epoch: 10 [550/13500]	Loss: 2370.9565
Training Epoch: 10 [600/13500]	Loss: 2469.1299
Training Epoch: 10 [650/13500]	Loss: 2368.4294
Training Epoch: 10 [700/13500]	Loss: 2366.8901
Training Epoch: 10 [750/13500]	Loss: 2410.0237
Training E

Training Epoch: 10 [8350/13500]	Loss: 2320.0581
Training Epoch: 10 [8400/13500]	Loss: 2309.5728
Training Epoch: 10 [8450/13500]	Loss: 2357.9058
Training Epoch: 10 [8500/13500]	Loss: 2292.5552
Training Epoch: 10 [8550/13500]	Loss: 2260.3796
Training Epoch: 10 [8600/13500]	Loss: 2275.6338
Training Epoch: 10 [8650/13500]	Loss: 2329.7280
Training Epoch: 10 [8700/13500]	Loss: 2315.5801
Training Epoch: 10 [8750/13500]	Loss: 2299.7634
Training Epoch: 10 [8800/13500]	Loss: 2292.6897
Training Epoch: 10 [8850/13500]	Loss: 2217.0566
Training Epoch: 10 [8900/13500]	Loss: 2293.3931
Training Epoch: 10 [8950/13500]	Loss: 2326.4270
Training Epoch: 10 [9000/13500]	Loss: 2289.1804
Training Epoch: 10 [9050/13500]	Loss: 2311.6431
Training Epoch: 10 [9100/13500]	Loss: 2347.8723
Training Epoch: 10 [9150/13500]	Loss: 2339.2385
Training Epoch: 10 [9200/13500]	Loss: 2258.1140
Training Epoch: 10 [9250/13500]	Loss: 2304.9443
Training Epoch: 10 [9300/13500]	Loss: 2330.3857
Training Epoch: 10 [9350/13500]	Loss: 23

Training Epoch: 11 [3300/13500]	Loss: 2246.4783
Training Epoch: 11 [3350/13500]	Loss: 2218.5554
Training Epoch: 11 [3400/13500]	Loss: 2104.5820
Training Epoch: 11 [3450/13500]	Loss: 2176.5803
Training Epoch: 11 [3500/13500]	Loss: 2248.6343
Training Epoch: 11 [3550/13500]	Loss: 2221.6094
Training Epoch: 11 [3600/13500]	Loss: 2198.5286
Training Epoch: 11 [3650/13500]	Loss: 2256.2241
Training Epoch: 11 [3700/13500]	Loss: 2182.5251
Training Epoch: 11 [3750/13500]	Loss: 2282.4326
Training Epoch: 11 [3800/13500]	Loss: 2250.0652
Training Epoch: 11 [3850/13500]	Loss: 2237.0452
Training Epoch: 11 [3900/13500]	Loss: 2212.2236
Training Epoch: 11 [3950/13500]	Loss: 2225.5266
Training Epoch: 11 [4000/13500]	Loss: 2200.2952
Training Epoch: 11 [4050/13500]	Loss: 2206.3984
Training Epoch: 11 [4100/13500]	Loss: 2214.3113
Training Epoch: 11 [4150/13500]	Loss: 2278.9395
Training Epoch: 11 [4200/13500]	Loss: 2292.3862
Training Epoch: 11 [4250/13500]	Loss: 2231.6160
Training Epoch: 11 [4300/13500]	Loss: 21